In [1]:
import os
import glob
import wandb
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.utils import set_determinism
from monai.data import CacheDataset, DataLoader, pad_list_data_collate

from utils.feta_utils import AddClassChannels
import time
from monai.data.utils import decollate_batch
from monai.transforms import EnsureType, AsDiscrete
from monai.inferers import sliding_window_inference
from monai.networks.utils import one_hot

In [2]:
data_dir = '/teamspace/uploads'

In [3]:
train_images = sorted(
    glob.glob(os.path.join(data_dir, "*T2w.nii.gz")))
train_labels = sorted(
    glob.glob(os.path.join(data_dir, "*dseg.nii.gz")))
data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]
train_files, val_files = data_dicts[:-5], data_dicts[-5:]

In [36]:
import torch
import torch.nn.functional as F
import wandb
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.transforms import AsDiscrete
from monai.inferers import sliding_window_inference
import numpy as np


gpu_device = "cuda:0" if torch.cuda.is_available else "cpu"
print(gpu_device)
device = torch.device("cuda:0")  # or "cpu"
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=8,  # 8 classes including background
    channels=(16, 32, 64, 256),
    strides=(2, 2, 2, 2),
    norm=Norm.BATCH,
).to(device)

# Initialize Wandb
wandb.init(project="FETA-model-training")

# Loss and metric configurations
loss_function = DiceLoss(
    smooth_nr=0, 
    squared_pred=True, 
    to_onehot_y=False,  
    softmax=False
)

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# Post-processing transforms
post_label = AsDiscrete()  # 
post_pred = AsDiscrete()

cuda:0


`len(strides) > len(channels) - 1`, the last 1 values of strides will not be used.


In [37]:
# train_transforms = monai.transforms.Compose(
#     [
#         monai.transforms.LoadImaged(keys=["image", "label"]),
#         monai.transforms.EnsureChannelFirstd(keys=["image", "label"]),
#         monai.transforms.ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=(128, 128, 128)),
#         monai.transforms.Orientationd(keys=["image", "label"], axcodes="RAS"),
#         monai.transforms.AdjustContrastd(keys=["image"], gamma=1.5),
#         monai.transforms.RandShiftIntensityd(keys=["image"], offsets=1, prob=0.5),
#         monai.transforms.RandAffined(
#             keys=["image", "label"], 
#             translate_range=(15, 20, 30), 
#             scale_range=(0.1, 0.1, 0.1), 
#             mode=["bilinear", "nearest"], 
#             prob=0.5
#         ),
#         monai.transforms.RandGaussianNoised(keys=["image"], mean=2, std=0.1, prob=0.5),
#         monai.transforms.RandAdjustContrastd(keys=["image"], gamma=(0.7, 1.3), prob=0.5),
#         monai.transforms.RandZoomd(keys=["image"], min_zoom=1.0, max_zoom=1.5, prob=0.4),
#         monai.transforms.NormalizeIntensityd(keys=["image"], nonzero=True, channel_wise=True)
#     ]
# )
train_transforms = monai.transforms.Compose(
    [
        monai.transforms.LoadImaged(keys=["image", "label"]),
        monai.transforms.EnsureChannelFirstd(keys=["image", "label"]),
        monai.transforms.ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=(128, 128, 128)),
        monai.transforms.Orientationd(keys=["image", "label"], axcodes="RAS"),
        monai.transforms.AdjustContrastd(keys=["image"], gamma=1.5),
        monai.transforms.NormalizeIntensityd(keys=["image"], nonzero=True, channel_wise=True)
    ]
)

val_transforms = monai.transforms.Compose(
    [
        monai.transforms.LoadImaged(keys=["image", "label"]),
        monai.transforms.EnsureChannelFirstd(keys=["image", "label"]),
        monai.transforms.ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=(128, 128, 128)),
        monai.transforms.Orientationd(keys=["image", "label"], axcodes="RAS"),
        monai.transforms.NormalizeIntensityd(keys=["image"], nonzero=True, channel_wise=True)
    ]
)

def log_transforms_to_wandb(train_transforms, val_transforms):
    train_transform_details = [str(transform) for transform in train_transforms.transforms]
    val_transform_details = [str(transform) for transform in val_transforms.transforms]
    
    wandb.log({
        "train_transforms": train_transform_details,
        "val_transforms": val_transform_details
    })
log_transforms_to_wandb(train_transforms, val_transforms)

In [11]:
train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0, num_workers=2)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2,collate_fn=pad_list_data_collate)

val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=2,collate_fn=pad_list_data_collate)

Loading dataset: 100%|██████████| 5/5 [00:03<00:00,  1.42it/s]


In [12]:
#check for shape incosistencies
from monai.utils import first

first_batch  = first(train_loader)
print(first_batch['image'].shape)
print(first_batch['label'].shape)

torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128])


In [38]:
# EarlyStopping Class
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.patience_counter = 0

    def should_stop(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.patience_counter = 0
        else:
            self.patience_counter += 1

        return self.patience_counter >= self.patience

In [39]:
# Validation Function
def validation(model, val_loader, loss_function, device, epoch):
    model.eval()
    val_loss = 0
    
    # Initialize classwise dice metrics
    class_dice_metrics = {
        f"class_{i}": DiceMetric(include_background=False, reduction="mean")
        for i in range(8)  # 8 classes
    }
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}", unit="batch")
        for val_data in progress_bar:
            val_images, val_labels = val_data["image"].to(device), val_data["label"].to(device)

            val_labels = val_labels.squeeze(1)
            val_labels_one_hot = F.one_hot(val_labels.long(), num_classes=8)
            val_labels_one_hot = val_labels_one_hot.permute(0, 4, 1, 2, 3).float()

            outputs = model(val_images)
            loss = loss_function(outputs, val_labels_one_hot)
            val_loss += loss.item()

            # Calculate classwise Dice scores
            for class_idx in range(8):
                class_key = f"class_{class_idx}"
                class_dice_metrics[class_key](
                    y_pred=[outputs[:, class_idx:class_idx+1]], 
                    y=[val_labels_one_hot[:, class_idx:class_idx+1]]
                )

            progress_bar.set_postfix(loss=loss.item())

    # Compute average loss and classwise Dice scores
    val_loss = val_loss / len(val_loader)
    
    # Calculate and log classwise dice scores
    val_dice_scores = {}
    mean_dice = 0
    for class_idx in range(8):
        class_key = f"class_{class_idx}"
        dice_val = class_dice_metrics[class_key].aggregate().item()
        
        # Ensure dice values are between 0 and 1
        if dice_val > 1 or dice_val < 0:
            dice_val = abs(dice_val % 1)
            
        val_dice_scores[f"val_dice_{class_key}"] = dice_val
        mean_dice += dice_val

    # Calculate mean dice across all classes
    mean_dice = mean_dice / 8

    # Log all metrics to wandb
    wandb.log({
        "validation_loss": val_loss,
        "validation_dice": mean_dice,
        "epoch": epoch + 1,
        **val_dice_scores  # Unpack all classwise dice scores
    })

    print(f"[Validation] Epoch {epoch+1}: Loss: {val_loss:.4f}, Mean Dice: {mean_dice:.4f}")
    return val_loss, mean_dice

# Training Function
def train(model, global_step, train_loader, val_loader, loss_function, optimizer, device, dice_val_best, global_step_best, epoch, early_stopping):
    model.train()
    epoch_loss = 0
    
    # Initialize classwise dice metrics
    class_dice_metrics = {
        f"class_{i}": DiceMetric(include_background=False, reduction="mean")
        for i in range(8)  # 8 classes
    }

    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch + 1}", unit="batch")
    for step, train_data in enumerate(progress_bar):
        images, labels = train_data["image"].to(device), train_data["label"].to(device)

        labels = labels.squeeze(1)
        labels_one_hot = F.one_hot(labels.long(), num_classes=8).permute(0, 4, 1, 2, 3).float()

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels_one_hot)
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

        # Update classwise metrics
        for class_idx in range(8):
            class_key = f"class_{class_idx}"
            class_dice_metrics[class_key](
                y_pred=[outputs[:, class_idx:class_idx+1]], 
                y=[labels_one_hot[:, class_idx:class_idx+1]]
            )

        if step % 10 == 0:
            progress_bar.set_postfix(loss=loss.item())

    # Calculate and log training metrics
    epoch_loss = epoch_loss / len(train_loader)
    
    # Calculate classwise dice scores
    train_dice_scores = {}
    mean_train_dice = 0
    for class_idx in range(8):
        class_key = f"class_{class_idx}"
        dice_val = class_dice_metrics[class_key].aggregate().item()
        
        if dice_val > 1 or dice_val < 0:
            dice_val = abs(dice_val % 1)
            
        train_dice_scores[f"train_dice_{class_key}"] = dice_val
        mean_train_dice += dice_val

    mean_train_dice = mean_train_dice / 8

    # Log all metrics to wandb
    wandb.log({
        "train_loss": epoch_loss,
        "train_dice":mean_train_dice,
        "epoch": epoch + 1,
        **train_dice_scores  # Unpack all classwise dice scores
    })

    print(f"[Training] Epoch {epoch + 1}: Loss: {epoch_loss:.4f}, Mean Dice: {mean_train_dice:.4f}")

    # Validation
    val_loss, val_dice = validation(model, val_loader, loss_function, device, epoch)

    # Early stopping check
    stop_training = early_stopping.should_stop(val_loss)
    if stop_training:
        print("Early stopping triggered")
        return global_step, dice_val_best, global_step_best, stop_training

    if val_dice > dice_val_best:
        dice_val_best = val_dice
        global_step_best = global_step

    global_step += 1
    return global_step, dice_val_best, global_step_best, stop_training

In [40]:
max_epochs = 250
early_stopping = EarlyStopping(patience=15, min_delta=0.0001)
global_step = 0
dice_val_best = 0.0
global_step_best = 0
# Training Loop
for epoch in range(max_epochs):
    global_step, dice_val_best, global_step_best, stop_training = train(
        model, 
        global_step, 
        train_loader, 
        val_loader, 
        loss_function, 
        optimizer, 
        device,
        dice_val_best, 
        global_step_best, 
        epoch, 
        early_stopping
    )
    
    if stop_training:
        break
# Finish Wandb logging
wandb.finish()

Training Epoch 1: 100%|██████████| 38/38 [00:07<00:00,  5.08batch/s, loss=0.883]


[Training] Epoch 1: Loss: 0.9275, Mean Dice: 0.1499


Validation Epoch 1: 100%|██████████| 5/5 [00:00<00:00,  6.40batch/s, loss=0.859]


[Validation] Epoch 1: Loss: 0.8911, Mean Dice: 0.1223


Training Epoch 2: 100%|██████████| 38/38 [00:07<00:00,  5.11batch/s, loss=0.818]


...

Training Epoch 250: 100%|██████████| 38/38 [00:07<00:00,  4.85batch/s, loss=0.225]


[Training] Epoch 250: Loss: 0.1383, Mean Dice: 0.7199


Validation Epoch 250: 100%|██████████| 5/5 [00:00<00:00,  6.04batch/s, loss=0.107]


[Validation] Epoch 250: Loss: 0.1978, Mean Dice: 0.7745
Early stopping triggered


epoch,▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█
train_dice,▁▆▅▆▅▆▅▆▆▆▅▆▇▆▇▇▆▇▇▇▇▇▇█▆█▇▇▇▆█▆█▇▇▆▇█▇▆
train_dice_class_0,▁▃▄▆▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇██▇██▇████▇███████▇██
train_dice_class_1,▁▂▂▂▃▄▄▃▄▄▃▄▄▄▅▄▆▅▆▆▆▆▇▆▇▃█▇▇▆▇▆▇▇▃▇▇▆▆▅
train_dice_class_2,▃▃▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██▁▇▇▇███████████████
train_dice_class_3,▁▄▄▅▅▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇███▇▇▇▇▇██▇███
train_dice_class_4,▂▂▁▄▅▅▅▅▆▆▆▆▆▆▇▆▅▅▆▇▇█▆▅▆▆▇▇▆▆▇▇▇▇▇▆▇▇▆▇
train_dice_class_5,▁▁▁▂▄▆▅▅▅▇▆▇▆▁▆▅▇▇▅▅▇▇█▆▅▆▇▄▆█▁▇▄▄██▇█▁▂
train_dice_class_6,▂▂▃▅▆▆▆▆▆█▇▇▇▆▂▃▇▇▃▁▇▁▂▇▇▇█▇█▇▇▇████▂█▅▂
train_dice_class_7,▁▂▃▄▁▆▆▆▆▆▆▇▇▇▆▅▇▇█▇▇▇▇▃▅▆▆█▃▆▆▆▅▇▇▇▇▇█▇
train_loss,█▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
